In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from POT import POT

In [2]:
def split_pos_neg(p):
    p_pos_full = np.zeros_like(p)#, dtype=np.float16)
    p_neg_full = np.zeros_like(p)#, dtype=np.float16)

    p_pos_indices = np.where(p > 0)[0]
    p_neg_indices = np.where(p < 0)[0]

    p_pos_full[p_pos_indices] = p[p_pos_indices]
    p_neg_full[p_neg_indices] = p[p_neg_indices]

    p_neg_full = np.abs(p_neg_full)

    return p_pos_full, p_neg_full

In [3]:
# Load pressure and DOA data for locations 2 and 4
pressure_files = [r'C:\Users\r0821191\Documents\helixon\SRIR40k\ground_1_0_p.csv', r'C:\Users\r0821191\Documents\helixon\SRIR40k\ground_5_0_p.csv']
doa_files = [r'C:\Users\r0821191\Documents\helixon\SRIR40k\ground_1_0_doa.csv', r'C:\Users\r0821191\Documents\helixon\SRIR40k\ground_5_0_doa.csv']

p_w_pos = []
p_w_neg = []
doa_data = []

for i in range(len(pressure_files)):
    df_pressure = pd.read_csv(pressure_files[i], header=None).values.flatten()#.astype(np.float16)
    df_pressure = df_pressure[400:40000]#.astype(np.float16)
    print("PRESSURE SHAPE: ")
    print(df_pressure.shape)
    print(df_pressure[0] )
    print()


    df_doa = pd.read_csv(doa_files[i], header=None).values#.astype(np.float16)
    df_doa = df_doa[400:40000]#.astype(np.float16)
    print("DOA SHAPE: ")
    print(df_doa.shape)
    print(df_doa[0])
    print()

    combined_df = pd.DataFrame({'pressure': df_pressure, 'doa_x': df_doa[:,0], 'doa_y': df_doa[:,1], 'doa_z': df_doa[:,2]})

    # Drop rows with any NaN values and get corresponding indices
    combined_df = combined_df.dropna()

    # Separate pressure data back
    df_pressure = combined_df['pressure'].values
    df_doa = combined_df[['doa_x', 'doa_y', 'doa_z']].values



    # Splitting positive and negative pressure
    p_pos, p_neg = split_pos_neg(df_pressure)
    p_w_pos.append(p_pos)
    p_w_neg.append(p_neg)
    doa_data.append(df_doa)

doa_data[0] = doa_data[0][0:30000]
doa_data[1] = doa_data[1][0:30000]
p_w_pos[0] = p_w_pos[0][0:30000]
p_w_pos[1] = p_w_pos[1][0:30000]
p_w_neg[0] = p_w_neg[0][0:30000]
p_w_neg[1] = p_w_neg[1][0:30000]




PRESSURE SHAPE: 
(39600,)
0.0314622997415602

DOA SHAPE: 
(39600, 3)
[-0.30038384 -0.90115151  2.70345452]

PRESSURE SHAPE: 
(39600,)
-0.0164291138301105

DOA SHAPE: 
(39600, 3)
[ 2.29442997e-16 -1.28148124e+00 -2.56296248e+00]



In [4]:
# Create point clouds for positive and negative pressures
PC2_pos = {'pos': doa_data[0].reshape(-1,3), 'mass': p_w_pos[0], 'n': len(doa_data[0])}
PC4_pos = {'pos': doa_data[1].reshape(-1,3), 'mass': p_w_pos[1], 'n': len(doa_data[1])}

PC2_neg = {'pos': doa_data[0].reshape(-1,3), 'mass': p_w_neg[0], 'n': len(doa_data[0])}
PC4_neg = {'pos': doa_data[1].reshape(-1,3), 'mass': p_w_neg[1], 'n': len(doa_data[1])}


print("CHECK IF DIMENSIONS ARE RIGHT: ")
print(PC2_neg['pos'].shape)
print(PC2_pos['mass'].shape)
print(PC2_pos['n'])
print()

CHECK IF DIMENSIONS ARE RIGHT: 
(30000, 3)
(30000,)
30000



In [5]:
# Set expected distance and tolerance
distEx = np.nanmean(np.abs(PC4_pos['pos'] - PC2_pos['pos']))#.astype(np.float16)  # IDK WHAT TO PUT SO I DID THIS
distTol = 0.2 #np.float16(0.2)
print("DIST EX: ")
print(distEx)
print()


DIST EX: 
73.8149854795818



In [6]:
pot_pos = POT(PC2_pos, PC4_pos, distEx, distTol)
print("REAACHEEEDDDDDDDDDDDDD end of pot pos")
print(np.mat(pot_pos.T).shape)

KeyboardInterrupt: 

In [7]:
pot_neg = POT(PC2_neg, PC4_neg, distEx, distTol)
print("REAACHEEEDDDDDDDDDDDDD end of pot neg")

COST MATRIX SHAPE: 
(30000, 30000)
DUMMY COST: 217.94608325403487
mass transported: 12708.336241443649 index: 0
NumIterMax: 10000000
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 24044
Cost coarse at index 0
mass transported: 12849.53997745969 index: 1
NumIterMax: 10000000
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 24544
Cost coarse at index 1
mass transported: 12990.743713475731 index: 2
NumIterMax: 10000000
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 25075
Cost coarse at index 2
mass transported: 13131.947449491772 index: 3
NumIterMax: 10000000
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 25572
Cost coarse at index 3
mass transported: 13273.151185507812 index: 4
NumIterMax: 10000000
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 26096
Cost coarse at index 4
mass transported: 13414.354921523853 index: 5
NumIterMax: 10000000
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 26616
Cost coarse at index 5
mass transported: 13555.55865

## END OF POT. INTERPOLATION OF PC BEGINS HERE

In [8]:
##PROLLY BETTER INTERPOLATION
k=0.5
PCk_pos_better = pot_pos.interpolatePC(k)
PCk_neg_better = pot_neg.interpolatePC(k)

## MAKE CSVs
This takes the format 
method_PC-Interpolated_fromwhat-towhat_pressure/doa_pos/neg.csv
e.g. pot_2_1-3_p_pos.csv, pot_2_1-3_doa_pos.csv

In [9]:
pd.DataFrame(PCk_pos_better['pressure']).to_csv('interp_csv/potMue003_3_1-5_p_pos.csv', header=False, index=False)
pd.DataFrame(PCk_pos_better['doa']).to_csv('interp_csv/potMue003_3_1-5_doa_pos.csv', header=False, index=False)

pd.DataFrame(PCk_neg_better['pressure']).to_csv('interp_csv/pot_3_1-5_p_neg.csv', header=False, index=False)
pd.DataFrame(PCk_neg_better['doa']).to_csv('interp_csv/pot_3_1-5_doa_neg.csv', header=False, index=False)

In [10]:
### combine pc
all_pressures = np.concatenate(PCk_pos['mass'], PCk_neg['mass'])
all_doas = np.concatenate(PCk_pos['pos'], PCk_neg['pos'])

NameError: name 'PCk_pos' is not defined

In [2]:
### RIR
c=243
timeshifts = np.linalg.norm(all_doas, axis=1)/c

time = np.linspace(0,1,100000)
ir = np.zeros_like(time)

for i in range (len(all_pressures)):
    index = int(np.searchsorted(time, timeshifts[i])
    ir[index]+= all_pressures[i]


plt.figure(figsize=(8,6))
plt.plot(time, ir)
plt.title('Ímpulse Response')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

NameError: name 'all_doas' is not defined